# Lab 4: AgentCore Browser を使用した予防エージェント

## 概要
AgentCore Browser を使用して、リアルタイムの AWS ドキュメント調査とプロアクティブなインフラストラクチャ推奨を行う Strands ベースの予防エージェントを構築します。

## 目標
- AgentCore Browser 統合を持つ Strands エージェントを作成
- 予防重視の分析ワークフローを実装
- リアルタイムの AWS ベストプラクティス調査用ツールを構築
- インフラストラクチャ予防シナリオでエージェントをテスト
- 問題を防ぐためのプロアクティブな推奨を生成

## 学習内容
- AgentCore Browser と Strands エージェントを統合する方法
- リアクティブな修復 vs 予防重視のワークフローを実装する方法
- ブラウザ自動化を使用して現在の AWS ベストプラクティスを調査する方法
- エージェントの予防分析と推奨パターン

## アーキテクチャ概要
```
┌─────────────────┐    ┌──────────────────────┐    ┌─────────────────────┐
│   ユーザーリクエスト │───▶│  Strands エージェント   │───▶│  AgentCore Browser  │
│   （予防）         │    │  （予防）              │    │                     │
└─────────────────┘    └──────────────────────┘    └─────────────────────┘
                                │                            │
                                ▼                            ▼
                       ┌──────────────────────┐    ┌─────────────────────┐
                       │  予防分析            │    │  リアルタイム Web    │
                       │  ├─ リスク評価       │    │  調査               │
                       │  ├─ ベストプラクティス│    │  ├─ AWS ドキュメント │
                       │  └─ 推奨事項         │    │  ├─ ベストプラクティス│
                       └──────────────────────┘    │  └─ 現在の標準       │
                                │                  └─────────────────────┘
                                ▼                            │
                       ┌──────────────────────┐              ▼
                       │  プロアクティブな     │    ┌─────────────────────┐
                       │  推奨事項            │◀───│  ドキュメント分析    │
                       │  （問題発生前に）     │    │  とインサイト        │
                       └──────────────────────┘    └─────────────────────┘
```

**主要コンポーネント:**
- **予防フォーカス**: リアクティブな修正 vs プロアクティブな推奨
- **リアルタイム調査**: 現在の AWS ドキュメントへのライブアクセス
- **ブラウザ統合**: Web コンテンツアクセス用の AgentCore Browser
- **現在の標準**: 常に最新の AWS ベストプラクティスを使用

## 0. 必要なパッケージのインストール

最初にこのセルを実行して、すべての依存関係がインストールされていることを確認してください。

In [ ]:
%pip install -q -r requirements.txt
print("✅ Prevention agent dependencies installed")

## 1. 必要なモジュールのインポート

In [ ]:
# AWS SDK and configuration
import logging
from datetime import datetime

# Strands framework
from strands import Agent
from strands.models import BedrockModel
from strands.tools import tool
from strands_tools.browser import AgentCoreBrowser

# Workshop configuration
# Bedrock AgentCore Starter Toolkit
from bedrock_agentcore_starter_toolkit import Runtime
from lab_helpers.config import MODEL_ID, AWS_REGION, AWS_PROFILE, WORKSHOP_NAME
from lab_helpers.parameter_store import get_parameter, put_parameter
from lab_helpers.constants import PARAMETER_PATHS
from lab_helpers.lab_04.gateway_setup import AgentCoreGatewaySetup

# Configure logging for notebook
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
logger = logging.getLogger(__name__)

print("✅ Imports loaded")
print(f"   Workshop: {WORKSHOP_NAME}")
print(f"   Region: {AWS_REGION}")
print(f"   Model: {MODEL_ID}")
print("🌐 AgentCore Browser tool imported")
print("🔮 Prevention agent ready for setup")

## 2. 前提条件の確認

In [ ]:
# Verify prerequisites are available
try:
    # Test AWS credentials
    import boto3
    sts_client = boto3.client('sts', region_name=AWS_REGION)
    identity = sts_client.get_caller_identity()
    account_id = identity['Account']
    
    # Test AgentCore Browser availability
    browser_tool = AgentCoreBrowser(region=AWS_REGION)
    
    print(f"✅ Prerequisites verified: AWS Account {account_id}, AgentCore Browser available")
    print(f"   Region: {AWS_REGION}")
    print(f"   Profile: {AWS_PROFILE}")
    print(f"   Model ID: {MODEL_ID}")
    print(f"   Identity: {identity.get('Arn', 'Unknown')}")
    print(f"   Browser Tool: AgentCoreBrowser initialized")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("Please ensure AWS credentials are configured and AgentCore Browser permissions are available.")

## 3. Browser 統合を持つ予防エージェントの作成

**目標:** リアルタイム Web 調査用の AgentCore Browser を持つ Strands エージェントをセットアップ。

**アプローチ:** ブラウザツールを初期化し、予防重視のシステムプロンプトでエージェントを作成。

**主な学習ポイント:** AgentCore Browser が予防のためのリアルタイムドキュメント調査をどのように可能にするか。

In [ ]:
### 3.1: Setup Prevention Agent with Browser Tool

def setup_prevention_agent(region=AWS_REGION):
    """Setup Strands agent with AgentCore Browser tool"""
    try:
        # Initialize the Browser tool - following strands_tools.browser pattern
        browser_tool = AgentCoreBrowser(region=region)
        
        # Setup Bedrock model
        model = BedrockModel(
            model_id=MODEL_ID,
            streaming=True,
        )
        
        # Create agent with browser tool 
        agent = Agent(
            model=model, 
            tools=[browser_tool.browser]
        )
        
        logger.info("✅ SRE Prevention Agent ready with AgentCore Browser")
        logger.info(f"🌍 Region: {region}")
        logger.info("🌐 Browser tool: AgentCoreBrowser integrated")
        
        return agent
        
    except Exception as e:
        logger.error(f"❌ Failed to setup agent: {e}")
        return None

# Setup the agent
agent = setup_prevention_agent()
if agent:
    print("✅ Strands prevention agent created successfully")
    print(f"   Model: {MODEL_ID}")
    print(f"   Tools: 1 (AgentCore Browser)")
    print("🌐 AgentCore Browser integration ready")
    print("🔮 Ready for prevention analysis")
else:
    print("❌ Agent setup failed!")
    print("   Check AgentCore Browser initialization and AWS credentials")

## 4. ブラウザ統合のテスト

**目標:** 基本的な Web アクセステストで AgentCore Browser 機能を検証。

**アプローチ:** シンプルな AWS ホームページアクセスでブラウザツールをテスト。

**主な学習ポイント:** ブラウザ接続性とコンテンツ抽出を検証する方法。

In [ ]:
### 4.1: Test Browser Integration

# Setup the agent

agent = setup_prevention_agent()

def test_browser_integration(agent):
    """Test AgentCore Browser integration"""
    if not agent:
        print("❌ Agent not available for testing")
        return False
    
    print("🧪 Testing AgentCore Browser Integration...")
    print("=" * 50)
    
    test_prompt = """
    Please use the browser tool to visit https://aws.amazon.com and get the page title
    
    
    This is just a test to verify the browser tool is working correctly.
    """
    
    try:
        start_time = datetime.now()
        response = agent(test_prompt)
        test_time = (datetime.now() - start_time).total_seconds()
        
        print(f"✅ Browser test completed in {test_time:.2f} seconds")
        
        # Display the response (truncate if too long)
        response_content = response.message.get('content', [])
        if response_content:
            for content in response_content:
                if isinstance(content, dict) and 'text' in content:
                    text = content['text']
                    if len(text) > 800:
                        print("\n📋 BROWSER TEST RESULTS (first 800 chars):")
                        print("-" * 30)
                        print(f"{text[:800]}...\n\n[Full output: {len(text)} chars]")
                    else:
                        print("\n📋 BROWSER TEST RESULTS:")
                        print("-" * 30)
                        print(text)
                elif isinstance(content, str):
                    print("\n📋 BROWSER TEST RESULTS:")
                    print("-" * 30)
                    print(content)
        
        return True
        
    except Exception as e:
        print(f"❌ Browser test failed: {e}")
        return False

# Run the browser test
if agent:
    print("💡 To run browser test, uncomment the line below:")
    test_result = test_browser_integration(agent)
    if test_result:
        print("\n🎉 AgentCore Browser integration test PASSED!")
    else:
        print("\n❌ AgentCore Browser integration test FAILED!")
    print("⚠️  Note: This will make a real web request to AWS homepage")
else:
    print("❌ Agent not available for browser testing")

## メモリからコンテキストを充実させる

診断情報でコンテキストを充実させるために、キュレートされたメモリから追加情報を取得しましょう。

In [ ]:
agent_memory_client = boto3.client("bedrock-agentcore", region_name=AWS_REGION)

memory_id = get_parameter(PARAMETER_PATHS['memory']['memory_id'])
memory_session_id = get_parameter(PARAMETER_PATHS['memory']['default_session_id'])

print(memory_id)
print(memory_session_id)
actor_id='diagnostics_agent'

  

#list events added to agent memory, to confirm successful write
params = {
                "memoryId": memory_id,
                "actorId": actor_id,
                "sessionId": memory_session_id,
                "includePayloads": True
            }
# Get all messages
response = agent_memory_client.list_events(**params)
additional_context=""
for event in response.get("events", []):
    payload = event.get('payload', [])
    for i, item in enumerate(payload):
        if 'conversational' in item:
            text = item['conversational']['content']['text']
            additional_context+=text
additional_context

## 5. 予防分析ワークフロー

**目標:** 実際の AWS ドキュメント調査を使用した予防分析ワークフローをデモンストレーション。

**アプローチ:** ブラウザを使用してベストプラクティスを調査し、包括的な予防分析を実行。

**主な学習ポイント:** 予防エージェントがプロアクティブな対策をどのように調査し推奨するか。

In [ ]:
### 5.1: Run Prevention Analysis

# Setup the agent

agent = setup_prevention_agent()

def analyze_prevention_opportunities(agent):
    """Analyze infrastructure for prevention opportunities"""
    if not agent:
        logger.error("❌ Agent not initialized")
        return None
    
    logger.info("🔮 Starting prevention opportunity analysis...")
    
    prevention_prompt = f"""
    I need you to analyze our CRM infrastructure for prevention opportunities using the browser tool to access AWS documentation. 
    Our CRM application is hosted on EC2 instances with a DynamoDB backend for customer data. 

    Here is additional information about the issues identified and fixes that have been applied:

    {additional_context}
    
    Please use the browser tool to access these specific AWS documentation pages and provide analysis:
    
    1. First, use the browser tool to visit: https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/best-practices.html
    2. Then visit: https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/ec2-best-practices.html  
    3. Finally visit: https://docs.aws.amazon.com/wellarchitected/latest/framework/
    
    Based on what you find in the AWS documentation, provide analysis focusing on:
    
    1. **Proactive Infrastructure Management**: Best practices we should implement
    2. **Customer Impact Prevention**: How to prevent issues that could affect customer experience  
    3. **Performance Optimization**: Latest AWS recommendations for EC2 and DynamoDB optimization
    4. **Monitoring and Alerting**: Best practices for proactive monitoring
    
    Provide your analysis with:
    - Executive summary of prevention opportunities
    - Customer impact prioritization  
    - Implementation roadmap with AWS best practices
    - Success metrics for measuring prevention effectiveness
    
    Use the browser tool to get the most current AWS documentation and recommendations.
    """
    
    try:
        start_time = datetime.now()
        response = agent(prevention_prompt)
        analysis_time = (datetime.now() - start_time).total_seconds()
        
        logger.info(f"✅ Prevention analysis completed in {analysis_time:.2f} seconds")
        
        return {
            'analysis_time': analysis_time,
            'response': response,
            'timestamp': datetime.now().isoformat()
        }
        
    except Exception as e:
        logger.error(f"❌ Prevention analysis failed: {e}")
        return None

# Run the complete prevention analysis
if agent:
    print("🚀 Starting Complete Prevention Analysis Workflow...")
    print("=" * 60)
    print("🔮 This will research current AWS best practices using the browser")
    print("📚 Accessing live AWS documentation for latest recommendations")
    print("⏳ This may take a few minutes as we browse multiple documentation pages...")
    result = analyze_prevention_opportunities(agent)
    
    if result:
        print(f"\n🎯 PREVENTION ANALYSIS RESULTS:")
        print(f"Analysis Time: {result['analysis_time']:.2f} seconds")
        print(f"Timestamp: {result['timestamp']}")
         
         # Display the agent's response
        response_content = result['response'].message.get('content', [])
        if response_content:
            for content in response_content:
                if isinstance(content, dict) and 'text' in content:
                    text = content['text']
                    if len(text) > 2000:
                        print(f"\n📋 PREVENTION ANALYSIS (first 2000 chars):\n{text[:2000]}...")
                    else:
                        print(f"\n📋 PREVENTION ANALYSIS:\n{text}")
                elif isinstance(content, str):
                    print(f"\n📋 PREVENTION ANALYSIS:\n{content}")
         
        print("\n🎉 Prevention analysis workflow completed successfully!")
    else:
        print("❌ Prevention analysis failed!")
    
    print("⚠️  Note: This will make multiple browser requests and may take several minutes")
    print("🔒 This demonstrates proactive analysis vs reactive remediation")
    
else:
    print("❌ Agent not available for prevention analysis!")

## 6. AgentCore Runtime へのデプロイ

**目標:** 予防エージェントを Amazon Bedrock AgentCore Runtime にデプロイしてサーバーレス実行。

**アプローチ:** AgentCoreRuntimeDeployer を使用して Runtime インフラストラクチャを作成し、エージェントをデプロイ。

**主な学習ポイント:** Browser 統合を持つ Strands エージェントを本番環境対応のサーバーレスインフラストラクチャにデプロイする方法。

### 6.1: Runtime Deployer の初期化と IAM ロールの作成

In [ ]:
from lab_helpers.lab_04 import AgentCoreRuntimeDeployer, store_runtime_configuration

# Initialize deployer
deployer = AgentCoreRuntimeDeployer(
    region=AWS_REGION,
    prefix=WORKSHOP_NAME,
    verbose=True
)

# Check prerequisites
print("✓ Checking prerequisites...")
if deployer.check_prerequisites():
    print("✅ All prerequisites met")
else:
    print("❌ Prerequisites not met")
    print("Please install: pip install fastmcp bedrock-agentcore-starter-toolkit strands-agents-tools")

# Create IAM role with Browser permissions
print("\n🔐 Creating IAM role for Runtime...")
role_info = deployer.create_runtime_iam_role()
print(f"   Role ARN: {role_info['role_arn']}")

### 6.2: AgentCore Runtime へのデプロイ

In [ ]:
### 6.2 a: Agent code for deployment

# Read the agent code file
with open('lab_helpers/lab_04/runtime_mcp_agent_code.py', 'r') as f:
    agentcore_agent_code = f.read()

print("✅ Agent code loaded from: lab_helpers/lab_04/runtime_mcp_agent_code.py")
print(f"✓ Loaded agent code: {len(agentcore_agent_code)} bytes")
print(f"✓ Code size: {len(agentcore_agent_code) / 1024:.1f} KB")
print(f"✓ AWS_REGION: {AWS_REGION}") 

In [ ]:
### 6.2 b: Write agent code to disk
with open('agent-prevention.py', 'w') as f:
      f.write(agentcore_agent_code)

print(f"✅ Agent code written: agent-prevention.py")

### 6.3 JWT オーソライザーで Runtime を構成

**runtime.configure() が行うこと:**
- エージェントコードと依存関係を検証
- Dockerfile と AWS 構成ファイルを生成
- デプロイメントブループリントを準備（ローカル操作、AWS リソースはまだ作成されない）
- 実行ロールとトークン検証をセットアップ

**JWT オーソライザー構成:**
`authorizer_configuration` パラメータは、Runtime に受信トークンの検証方法を伝えます：
- **discoveryUrl**: Runtime が署名検証用の公開鍵を取得する Cognito OIDC エンドポイント
- **allowedClients**: User Auth クライアント（直接ユーザー）と M2M クライアント（Gateway）の両方が許可

**自動的な Runtime の動作:**
```
リクエスト内の Bearer トークン → 署名を検証 → 発行者（Cognito）を確認 → allowedClients 内のクライアント ID を確認 → 許可または拒否
```

In [ ]:
### 6.3 a: Retrieve Cognito configuration
from lab_helpers.parameter_store import get_parameter
from lab_helpers.constants import PARAMETER_PATHS

# Retrieve Cognito configuration from Lab-01 SSM Parameter Store
cognito_domain = get_parameter(PARAMETER_PATHS['cognito']['domain'])
user_pool_id = get_parameter(PARAMETER_PATHS['cognito']['user_pool_id'])
m2m_client_id = get_parameter(PARAMETER_PATHS['cognito']['m2m_client_id'])
user_auth_client_id = get_parameter(PARAMETER_PATHS['cognito']['user_auth_client_id'])

# Build Cognito discovery URL - Runtime uses this to fetch public keys for token validation
# Using Cognito domain from Parameter Store (more reliable than manual construction)
discovery_url = f"https://cognito-idp.{AWS_REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration"

print(f"✅ Cognito configuration retrieved")
print(f"   Discovery URL: {discovery_url}")
print(f"   Allowed Clients: User Auth + M2M")
print(f"   User Auth Client ID: {user_auth_client_id}")
print(f"   M2M Client ID: {m2m_client_id}")

In [ ]:
### 6.3 b: Configure Runtime with JWT Authorizer

# Initialize Runtime object
runtime = Runtime()

# JWT Authorizer Configuration
# - discoveryUrl: Cognito OIDC endpoint (Runtime automatically fetches public keys)
# - allowedClients: Both User Auth and M2M clients can call the Runtime
authorizer_config = {
    "customJWTAuthorizer": {
        "discoveryUrl": discovery_url,
        "allowedClients": [user_auth_client_id, m2m_client_id]
    }
}

# CRITICAL: Configure Runtime with JWT token validation
runtime.configure(
      entrypoint='agent-prevention.py',
      execution_role=role_info['role_arn'],
      auto_create_ecr=True,
      requirements_file='requirements.txt',
      region=AWS_REGION,
      agent_name=f"{WORKSHOP_NAME}_prevention_runtime",
      protocol="MCP",
      authorizer_configuration=authorizer_config  # ← TOKEN VALIDATION ENABLED
)

print("✅ Runtime configured with JWT authorizer")
print(f"   Protocol: MCP")
print(f"   JWT Token Validation: ENABLED")
print(f"   Allowed Tokens: User Auth + M2M")

### 6.4: AgentCore に Runtime を起動

Python SDK `runtime.launch()` を使用して、構成済みの Runtime を AgentCore にデプロイします。この重要なステップにより、ローカルエージェントが本番環境のサーバーレスサービスに変換されます。

#### 6.4 a: 起動プロセスの理解

  **runtime.launch() 中に行われること:**

  1️⃣ CodeBuild が Docker コンテナのビルドを開始

  2️⃣ requirements.txt からの依存関係がインストール

  3️⃣ イメージが Amazon ECR にプッシュ（自動作成）

  4️⃣ AgentCore が Runtime を MCP サービスとして登録
  
  5️⃣ CloudWatch ロギングが構成

  ⏱️ **典型的な所要時間:** 5-10 分

In [ ]:
# Added to handle run all cells scenario
import time
time.sleep(10)

In [ ]:
### 6.4 a: Execute runtime.launch()

print("\n🚀 Launching Runtime to AgentCore...\n")

try:
    # Launch Runtime with auto-update for conflicts
    # This is synchronous - waits for CodeBuild and initial health checks
    launch_result = runtime.launch(auto_update_on_conflict=True)
    
    # Extract deployment ARN
    runtime_arn = launch_result.agent_arn
    
    print(f"✅ Runtime launched successfully!")
    print(f"   Runtime ARN: {runtime_arn}")
    print(f"\n📝 Configuration stored for next sections:")
    
except Exception as e:   
    print(f"❌ Launch failed: {e}")
    print(f"\nTroubleshooting:")
    print(f"  • Check CodeBuild service limits")
    print(f"  • Verify ECR permissions in IAM role")
    print(f"  • Review CloudWatch logs for build errors")
    print(f"  • Ensure all dependencies in requirements.txt are correct")
    raise

In [ ]:
### 6.4 b: Storing Runtime Configuration

# Extract and store runtime configuration
runtime_arn = launch_result.agent_arn
runtime_id = getattr(launch_result, 'agent_id', None)

# Store configuration
from lab_helpers.lab_04 import store_runtime_configuration
store_runtime_configuration(runtime_arn, runtime_id, region=AWS_REGION, prefix=WORKSHOP_NAME)

print("\n✅ Runtime deployed and configured")
print(f"   ARN: {runtime_arn}")
print(f"   Ready for Gateway registration")

## 7. CloudWatch ロギングの構成

**目標:** Runtime コンテナログ用の CloudWatch Logs デリバリーをセットアップ。

**アプローチ:** configure_runtime_logging を使用してログデリバリーを自動的にセットアップ。

**主な学習ポイント:** AgentCore がオブザーバビリティのために CloudWatch とどのように統合されるか。

In [ ]:
from lab_helpers.lab_04.configure_logging import configure_runtime_logging

# Configure CloudWatch Logs Delivery
print("📊 Configuring CloudWatch Logs Delivery...")

logging_config = configure_runtime_logging(
    runtime_arn=runtime_arn,
    runtime_id=runtime_id,
    region=AWS_REGION,
    log_type="APPLICATION_LOGS"
)

print(f"\n✅ Logging configured:")
print(f"   Log Group: {logging_config['log_group_name']}")
print(f"   Status: {logging_config['delivery_status']}")

## 8. AgentCore Gateway のデプロイ

**目標:** Gateway インフラストラクチャを作成し、Runtime をターゲットとして登録。

**アプローチ:** Cognito JWT 認証で Gateway を作成し、次に OAuth2 M2M 認証情報で Runtime を登録。

**主な学習ポイント:** デュアルレイヤー認証（インバウンド JWT、アウトバウンド OAuth2 M2M）をセットアップする方法。

In [ ]:
# Retrieve Cognito credentials from Lab-01
user_auth_client_id = get_parameter(PARAMETER_PATHS['cognito']['user_auth_client_id'])
user_pool_id = get_parameter(PARAMETER_PATHS['cognito']['user_pool_id'])

# Build Cognito OIDC discovery URL
discovery_url = f"https://cognito-idp.{AWS_REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration"

# Use helper to create Gateway IAM role 
gateway_setup = AgentCoreGatewaySetup(region=AWS_REGION, prefix=WORKSHOP_NAME, verbose=False)
role_info = gateway_setup.create_gateway_service_role()
role_arn = role_info['role_arn']

# Create Gateway directly with boto3 (simple API call)
agentcore = boto3.client('bedrock-agentcore-control', region_name=AWS_REGION)

gateway_response = agentcore.create_gateway(
      name=f"aiml301-prevention-gateway",
      roleArn=role_arn,
      protocolType="MCP",
      authorizerType="CUSTOM_JWT",
      authorizerConfiguration={
          "customJWTAuthorizer": {
              "discoveryUrl": discovery_url,
              "allowedClients": [user_auth_client_id]
          }
      }
  )

gateway_id = gateway_response['gatewayId']
gateway_url = gateway_response['gatewayUrl']

# Store configuration
put_parameter(PARAMETER_PATHS['lab_04']['gateway_id'], gateway_id)
put_parameter(PARAMETER_PATHS['lab_04']['gateway_role_arn'], role_arn)

print("✅ Gateway deployed with JWT authorization")
print(f"   Gateway ID: {gateway_id}")
print(f"   Gateway URL: {gateway_url}")
print(f"   Auth Type: Cognito JWT")

### 8.1: M2M 認証で Runtime を Gateway ターゲットとして追加

  **目標:** OAuth2 M2M 認証で Runtime を Gateway ターゲットとして登録。

  **アーキテクチャ:**

```
  ユーザー（JWT）→ Gateway（JWT 検証）→ Runtime（M2M トークン）
                      ↓
              Gateway が自動的に:
              - GetResourceOauth2Token を呼び出し
              - Secrets Manager から認証情報を取得
              - Cognito から M2M アクセストークンを取得
              - リクエストに Bearer トークンを注入
                      ↓
              Authorization: Bearer {token} で Runtime を呼び出し
```
  **Gateway が M2M OAuth2 トークンを自動送信する仕組み:**

  1. **認証情報プロバイダーストレージ**
     - OAuth2 認証情報プロバイダーは `clientId` + `clientSecret` を AWS Secrets Manager に保存（暗号化）
     - プロバイダー ARN はこのセキュアなストレージ場所を指す

  2. **自動トークン取得フロー**
  ```
     credentialProviderConfigurations で Gateway ターゲットを作成
         ↓
     Gateway が Runtime ターゲットを呼び出す必要がある
         ↓
     Gateway が GetResourceOauth2Token API を呼び出し（自動、組み込み）
         ↓
     AgentCore Identity が Secrets Manager から client_id + client_secret を取得
         ↓
     AgentCore が Cognito トークンエンドポイントを呼び出し：
     POST /token
     Body: grant_type=client_credentials&client_id=...&client_secret=...&scope=...
         ↓
     Cognito が M2M アクセストークンを返却
         ↓
     AgentCore がトークンをキャッシュ + リフレッシュライフサイクルを管理
         ↓
     Gateway がリクエストヘッダーにトークンを注入：
     Authorization: Bearer {access_token}
         ↓
     Gateway が Bearer トークン付きで Runtime MCP エンドポイントを呼び出し
         ↓
     Runtime が Cognito 公開鍵（JWKS）を使用して JWT 署名を検証
```
  3. **カスタムコード不要**
  - すべての認証情報管理が自動
  - トークンリフレッシュが自動
  - トークン注入が自動
  - コードは認証情報プロバイダー ARN を指定するだけ

  **主な学習ポイント:** デュアル認証 - ユーザーベースのインバウンド（JWT）、サービスベースのアウトバウンド（OAuth2 M2M）。

In [ ]:
# Added to handle run all cells scenario
import time
time.sleep(10)

Lab-03 を正常に実行した場合、そこで作成した認証情報プロバイダーを再利用します。Lab-04 を直接実行している場合は、以下のコメントを解除してください

In [ ]:
### 8.1 Create AgentCore Identity CredentialsProvider for Secure Access
## Retrieve M2M credentials from Lab-01 Cognito setup
#m2m_client_id = get_parameter(PARAMETER_PATHS['cognito']['m2m_client_id'])
#m2m_client_secret = get_parameter(PARAMETER_PATHS['cognito']['m2m_client_secret'])
#user_pool_id = get_parameter(PARAMETER_PATHS['cognito']['user_pool_id'])

## Build Cognito OIDC discovery URL
#discovery_url = f"https://cognito-idp.{AWS_REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration"

# Initialize AgentCore client
#agentcore = boto3.client('bedrock-agentcore-control', region_name=AWS_REGION)

## Create OAuth2 Credential Provider
## Stores M2M credentials securely in Secrets Manager
#credential_provider_response = agentcore.create_oauth2_credential_provider(
#      name='aiml301-m2m-credentials',
#      credentialProviderVendor='CustomOauth2',
#      oauth2ProviderConfigInput={
#          'customOauth2ProviderConfig': {
#              'clientId': m2m_client_id,
#              'clientSecret': m2m_client_secret,
#              'oauthDiscovery': {
#                  'discoveryUrl': discovery_url
#              }
#          }
#      }
#  )

## Extract response
#oauth2_provider_arn = credential_provider_response['credentialProviderArn']
#client_secret_arn = credential_provider_response['clientSecretArn']['secretArn']

## Store in SSM for next sections
#put_parameter(PARAMETER_PATHS['lab_03']['oauth2_provider_arn'], oauth2_provider_arn)
#put_parameter(PARAMETER_PATHS['lab_03']['oauth2_secret_arn'], client_secret_arn)

#print("✅ OAuth2 Credential Provider created")
#print(f"\n📋 Credential Storage:")
#print(f"   Provider ARN: {oauth2_provider_arn}")
#print(f"   Secret ARN: {client_secret_arn}")
#print(f"   Location: AWS Secrets Manager (encrypted)")
#print(f"   Credentials: M2M client_id + client_secret")

### 8.2: OAuth2 認証情報プロバイダーの作成と Runtime ターゲットの登録

In [ ]:
# Get M2M credentials from Parameter Store
# Retrieve configurations from SSM  
gateway_id = get_parameter(PARAMETER_PATHS['lab_04']['gateway_id'], region_name=AWS_REGION)
runtime_arn = get_parameter(PARAMETER_PATHS['lab_04']['runtime_arn'], region_name=AWS_REGION)

# If using Lab-03 created credentials provider
oauth2_provider_arn = get_parameter(PARAMETER_PATHS['lab_03']['oauth2_provider_arn'], region_name=AWS_REGION)
# If not using Lab-03, uncomment below to create new credentials provider
#oauth2_provider_arn = get_parameter(PARAMETER_PATHS['lab_04']['oauth2_provider_arn'], region_name=AWS_REGION)

resource_server_id = get_parameter(PARAMETER_PATHS['cognito']['resource_server_identifier'], region_name=AWS_REGION)

In [ ]:
print(f"   Gateway ID: {gateway_id}")
print(f"   Gateway URL: {gateway_url}")
print(f"   Runtime ARN: {runtime_arn}")

In [ ]:
### Create Target for Runtime with M2M OAuth2

import urllib.parse

# IMP: Construct endpoint URL from Runtime ARN
# Format: https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{URL_ENCODED_ARN}/invocations?qualifier=DEFAULT
# Example ARN: arn:aws:bedrock-agentcore:us-west-2:123456789012:runtime/my-runtime
encoded_arn = urllib.parse.quote(runtime_arn, safe='')
endpoint_url = f"https://bedrock-agentcore.{AWS_REGION}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"

# M2M scopes for fine-grained authorization
m2m_scopes = [
    f"{resource_server_id}/mcp.invoke",
    f"{resource_server_id}/runtime.access"
]

# Initialize AgentCore client
agentcore = boto3.client('bedrock-agentcore-control', region_name=AWS_REGION)

# Create Gateway Target - Runtime as external MCP server with M2M OAuth2
# The Gateway will automatically:
# 1. Use oauth2_provider_arn to get M2M token from Cognito
# 2. Include Bearer token when calling Runtime
# 3. Runtime validates token and authorizes operation based on scopes
target_response = agentcore.create_gateway_target(
    gatewayIdentifier=gateway_id,
    name="aiml301-runtime-target",
    description="AgentCore Runtime with M2M OAuth2 authentication",
    targetConfiguration={
        "mcp": {
            "mcpServer": {
                "endpoint": endpoint_url  # ← Constructed from Runtime ARN with URL encoding
            }
        }
    },
    credentialProviderConfigurations=[
        {
            "credentialProviderType": "OAUTH",
            "credentialProvider": {
                "oauthCredentialProvider": {
                    "providerArn": oauth2_provider_arn,  # ← References OAuth2 credential provider from Section 9.1
                    "scopes": m2m_scopes
                }
            }
        }
    ]
)

target_id = target_response['targetId']
put_parameter(PARAMETER_PATHS['lab_04']['gateway_runtime_target'], target_id, region_name=AWS_REGION)

print("✅ Runtime added as Gateway target with M2M OAuth2")
print(f"   Target ID: {target_id}")
print(f"   Runtime ARN: {runtime_arn}")
print(f"   Endpoint: {endpoint_url}")
print(f"   Credential Provider: {oauth2_provider_arn}")


In [ ]:
import time

### 9.2 Check Target Status and Synchronize

# Wait for target READY
print("\n⏳ Waiting for target to be READY...")
for attempt in range(30):
      target_info = agentcore.get_gateway_target(
          gatewayIdentifier=gateway_id,
          targetId=target_id
      )
      status = target_info.get('status', 'UNKNOWN')

      if status == 'READY':
          print(f"✅ Target is READY")
          break
      if status == 'FAILED' or status == 'SYNCHRONIZE_UNSUCCESSFUL':
          print(f"❌ Target in ERROR state: {target_info.get('statusReasons', 'No error message')}")
          break
      time.sleep(5)

# Synchronize to discover tools
agentcore.synchronize_gateway_targets(
      gatewayIdentifier=gateway_id,
      targetIdList=[target_id]
  )

print("\n✅ Complete - Gateway will automatically manage M2M tokens")

  ##### 重要な注意事項: 
  Gateway ロールに必要なもの:

  - bedrock-agentcore:GetResourceOauth2Token - 認証情報プロバイダーから M2M トークンを取得
  - secretsmanager:GetSecretValue - 保存された OAuth2 認証情報にアクセス

  なぜ？ Gateway が MCP 接続を Runtime に開始し、OAuth2 M2M 認証情報で認証する必要があるため。

  Runtime ロールに必要なもの:

  - bedrock-agentcore:GetWorkloadAccessToken - ワークロード ID を検証
  - bedrock-agentcore:CreateWorkloadIdentity - 自身の ID を確立
  - bedrock-agentcore:GetResourceOauth2Token - Gateway からの受信 OAuth2 トークンを検証

  なぜ？ Runtime は OAuth2 Bearer トークンを含む Gateway からの MCP リクエストを受信し、それらのトークンを検証して受信 MCP 呼び出しを認証する必要があるため。

## 9. MCP クライアントでデプロイされたエージェントをテスト

**目標:** JWT 認証で Gateway を通じて接続し、デプロイされたエージェントが動作することを確認。

**アプローチ:** Cognito で認証し、MCPClient を Gateway に接続し、予防ツールを呼び出す。

**主な学習ポイント:** MCP プロトコルを使用して本番エージェントをテストする方法。

### 9.1: Cognito で認証して JWT トークンを取得

In [ ]:
import boto3
from urllib.parse import urlencode
import requests
import json

# Get test user credentials
test_user_email = get_parameter(PARAMETER_PATHS['cognito']['test_user_email'])
test_user_password = get_parameter(PARAMETER_PATHS['cognito']['test_user_password'])

print("🔐 Authenticating with Cognito...")
print(f"   User: {test_user_email}")

cognito_client = boto3.client('cognito-idp', region_name=AWS_REGION)

# Authenticate user
auth_response = cognito_client.initiate_auth(
    ClientId=user_auth_client_id,
    AuthFlow='USER_PASSWORD_AUTH',
    AuthParameters={
        'USERNAME': test_user_email,
        'PASSWORD': test_user_password
    }
)

access_token = auth_response['AuthenticationResult']['AccessToken']
print(f"✅ JWT token obtained")
print(f"   Token length: {len(access_token)} chars")

In [ ]:
print((access_token))

### 9.2: MCPClient を接続してツールを一覧表示

In [ ]:
from lab_helpers.lab_04.mcp_client import MCPClient

# Connect MCP client to gateway
print("🔌 Connecting to Gateway via MCPClient...")
client = MCPClient(gateway_url, access_token, timeout=300)

# Initialize MCP session
print("\n📡 Initializing MCP session...")
server_info = client.initialize()

# List available tools
print("\n🔧 Listing available tools...")
tools = client.list_tools()

print(f"\n✅ Found {len(tools)} tools:")
for i, tool in enumerate(tools, 1):
    print(f"   {i}. {tool['name']}")
    desc = tool.get('description', 'No description')
    first_line = desc.split('\n')[0][:80]
    print(f"      {first_line}...")

In [ ]:
# Added to handle run all cells scenario
import time
time.sleep(10)

### 9.3: 予防分析ツールの呼び出し

In [ ]:
# Invoke analyze_infrastructure_prevention tool
print("🔮 Invoking prevention_agent tool...")

result = client.call_tool(
    "aiml301-runtime-target___research_agent",
    {
        "research_topic_query": f"""Research based on this {additional_context} for best practices to minimize issues in the future."""
    }
)

print(f"\n✅ Prevention analysis complete:")
if result['isError'] == False:

    print(result['content'][0]['text'])
    
else:
    print(f"   Error: {result}")

## 10. クリーンアップ

**目標:** すべての Lab-04 AWS リソースを削除。

**アプローチ:** cleanup_lab_04 を使用して Gateway、Runtime、IAM ロール、ログを削除。

**注意:** 継続的な課金を避けるため、テストが完了したらこれを実行してください。

In [ ]:
from lab_helpers.lab_04 import cleanup_lab_04

# Clean up all Lab-04 resources
print("🧹 Cleaning up Lab-04 resources...")
print("\nWARNING: This will delete:")
print("  • AgentCore Gateway and all targets")
print("  • AgentCore Runtime")
print("  • OAuth2 Credential Provider")
print("  • IAM roles")
print("  • CloudWatch logs")
print("\nTo proceed, uncomment the line below:")
print("# cleanup_lab_04(region_name=AWS_REGION, verbose=True)")

# Uncomment to run cleanup:
#cleanup_lab_04(region_name='us-west-2', verbose=True)

## サマリー: Lab 4 - 本番デプロイを伴う予防エージェント

✅ **完了:**
1. ✓ Browser 統合を持つ Strands 予防エージェントを作成
2. ✓ Browser Web アクセスとコンテンツ抽出をテスト
3. ✓ 予防分析ワークフローを実行（ローカル）
4. ✓ AgentCore Runtime にエージェントをデプロイ
5. ✓ CloudWatch Logs デリバリーを構成
6. ✓ Cognito JWT 認証で Gateway をデプロイ
7. ✓ OAuth2 M2M で Runtime を MCP ターゲットとして登録
8. ✓ JWT 認証で MCPClient 経由でテスト

**アーキテクチャ:**
```
予防エージェント（Strands + Browser）
    ↓
AgentCore Runtime（FastMCP）
    ↓
Gateway（インバウンド JWT 認証、アウトバウンド OAuth2 M2M）
    ↓
MCPClient（JWT でテスト）
```

**主な機能:**
- **プロアクティブな予防**: 予防機会のためのインフラストラクチャ分析
- **調査ベース**: 現在の AWS ベストプラクティスに AgentCore Browser を使用
- **本番環境対応**: サーバーレス、自動スケーリング、オブザーバブル
- **セキュア**: デュアルレイヤー認証（JWT + OAuth2）

**3 つの MCP ツールが利用可能:**
1. `analyze_infrastructure_prevention` - 予防機会を調査
2. `research_aws_best_practices` - AWS トピックを調査
3. `validate_prevention_environment` - 環境の準備状態を検証

**次のステップ:**
- Lab-05: マルチエージェントオーケストレーション（修復 + 予防）
- Lab-03 + Lab-04 を統一された SRE 自動化パイプラインに統合
- 調査 → 予防 → 修復 ワークフロー